In [32]:
import tensorflow as tf
import shutil
import numpy as np
import pandas as pd
import cv2
import os

class TFRecordExtractor:

    def __init__(self, tfrecord_file):
        self.tfrecord_file = os.path.abspath(tfrecord_file)
        self.count = 0

    def _extract_fn(self,tfrecord):

        # Extract features using the keys set during creation
        feature = {'label': tf.FixedLenFeature([], tf.int64),
                'label_normal': tf.FixedLenFeature([], tf.int64),
                'image': tf.FixedLenFeature([], tf.string)}


        # Decode the record read by the reader
        features = tf.parse_single_example(tfrecord, features=feature)
        # Convert the image data from string back to the numbers
        image = tf.decode_raw(features['image'], tf.uint8)
        #image = tf.image.decode_png(features['image'],channels=1)

        label = features['label']

        label_normal = features['label_normal']

        image = tf.reshape(image, [299, 299, 1])

        return [image,label,label_normal]


    def post_process_images(self):

        image_data_list = self.get_images()

        b_c_df = pd.DataFrame(columns=['id', 'class', 'normal_class'])

        id_list = []
        class_list = []
        normal_class_list = []

        for image_data in image_data_list:
            self.count = self.count + 1

            file_name = 'folder directory for writing images' + 'b_c' + str(
                self.count)
            id_list.append('b_c' + str(
                self.count))

            class_list.append(image_data[1])
            normal_class_list.append(image_data[2])


            cv2.imwrite(file_name+".png",image_data[0])

        id_arr = np.array(id_list)
        class_arr = np.array(class_list)
        normal_class_arr = np.array(normal_class_list)

        b_c_df["id"]  = pd.Series(id_arr)
        b_c_df["class"] = pd.Series(class_arr)
        b_c_df["normal_class"] = pd.Series(normal_class_arr)

        b_c_df.to_csv(
            path_or_buf=r"C:\DDSM\training_1.csv',
            columns=['id', 'class', 'normal_class'])

    def get_images(self):
        # Initialize all tfrecord paths

        dataset = tf.data.TFRecordDataset([self.tfrecord_file])
        dataset = dataset.map(self._extract_fn)
        iterator = dataset.make_one_shot_iterator()
        next_element = iterator.get_next()



        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            image_data_list = []
            try:
                while True:

                    image_data = sess.run(next_element)
                    image_data_list.append(image_data)

            except:
                pass
            
            return image_data_list

In [33]:
file = r"C:\training10_1\training10_1.tfrecords"
file2 = r"C:\training10_0\training10_0.tfrecords"
label,image = read_and_decode_single_example([file,file2])

In [40]:
t = TFRecordExtractor(file) 
t.post_process_images()

In [42]:
t